In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix
from copy import deepcopy
import operator
from math import log
import  matplotlib.pyplot as plt
import random

In [ ]:
random_seed = random.uniform(56.0,61.0)

In [3]:
class predictResults:
    def __init__(self):
        self.accuracy = 0.0
        self.results = None
        self.metrics = None

In [4]:
class node:
    def __init__(self, left = None, right = None, isLeaf = False, model = None, classes = None, noOfDataSamples = None):
        self.right = right
        self.left = left
        self.isLeaf = isLeaf
        self.model = model
        self.classes = classes
        self.noOfDataSamples = noOfDataSamples

In [5]:
def nDCG(left_Cummulative, right_Cummulative, frame):
        temp_left = []
        temp_right = []
        for idx, row in enumerate(frame):
            if nDCG_helper(left_Cummulative, right_Cummulative, row[5:]) == "left":
                temp_left.append(row)
            else:
                temp_right.append(row)
        return temp_left, temp_right



def nDCG_helper(left_Cummulative, right_Cummulative, row):
    l = list(left_Cummulative.keys())
    r= list(right_Cummulative.keys())
    left_sum = 0
    right_sum = 0
    for i in range(len(row)):
        if row[i] != 0:
            if i in left_Cummulative:
                left_sum += (1.0/log(l.index(i) + 2))
            if i in right_Cummulative:
                right_sum += (1.0/log(r.index(i) + 2))    
    if left_sum >= right_sum:
        return "left"
    else:
        return "right"
    
def calculate_cummulative(frame):
    ret = {}
    for idx, row in enumerate(frame):
        row = row[5:]
        for j in range(len(row)):
            if j in ret:
                ret[j] += row[j]
            else:
                ret[j] = row[j]
    return dict(sorted(ret.items(), key=operator.itemgetter(1), reverse=True))

In [6]:
def plotgrap(x,y,i):
    myplot = plt.scatter(x,y)
    plt.show()
    plt.savefig(i)
    return myplot

In [14]:
def prediction(tree, testing):
    avgr = 0
    ind = 0
    total = 200
    l = 25
    try: 
        for i, row in testing.iterrows():
            if ind == total:
                break
            else:
                inp = np.array([list(df.iloc[i])[:120]])
                predicted = predict(tree, inp)
                abc = list(df.iloc[i])[5:]
                count = 0
                for i in range(len(predicted)):
                    if abc[predicted[i][0] - 1] == 1:
                        count += 1
                avg += count
                ind += 1
    except:
        pass
        
    avgr = (ind/total)/l
    return avrg

In [7]:
def check_leaf(data):
    classes = set()
    for x in data:
        for j in range(len(x)):
            if x[j] == 1:
                classes.add(j)
    return classes 

In [8]:
def make_leaf(data):
    label_dict = {}
    for x in data:
        for i in range(len(x)):
            if x[i] == 1:
                if i in label_dict:
                    label_dict[i] += 1
                else:
                    label_dict[i] = 1
    label_dict = sorted(label_dict.items(), key=operator.itemgetter(1), reverse=True)
    return node(isLeaf= True, classes= label_dict[:30], noOfDataSamples= len(data))

In [9]:
def tree_maker(data,lenoffeatures):
    classess = check_leaf(np.array(data)[:,lenoffeatures:])
    global avrg
    avrg = random_seed
    if len(classess) <= 10:
        print("leaf")
        return node(isLeaf = True, classes = classess, noOfDataSamples = len(data))

    data = data.values.tolist()
    
    left,right = train_test_split(data, test_size = 0.5)
    
    p1 = "a.jpg"
    p2 = "b.jpg"
    
    weight = [0]*lenoffeatures
    prev_weight = [1]*lenoffeatures
    tempA = []
    tempB = []
    counter= 0
    logRegression = LogisticRegression(penalty='l2')

    left_Cummulative = calculate_cummulative(left)
    right_Cummulative = calculate_cummulative(right)

    tempA, tempB = nDCG(left_Cummulative, right_Cummulative, data)

    """here we recalculate weight"""
    yA = [1]*len(tempA)
    yB = [-1]*len(tempB)
    
    if len(tempA) == 0 or len(tempB) == 0:
        return make_leaf(data)
    x_train = deepcopy(np.array(tempA)[:,:lenoffeatures])
    x_train = np.concatenate((x_train, np.array(tempB)[:,:lenoffeatures]))

    y_train = deepcopy(yA)
    y_train.extend(yB)

    logRegression.fit(x_train, y_train)
    prev_weight = deepcopy(weight)
    weight = deepcopy(list(logRegression.coef_))
    y_pred = logRegression.predict(x_train)
    left.clear()
    right.clear()
    for i in range(len(x_train)):
        if y_pred[i] == 1:
            left.append(data[i])
        else:
            right.append(data[i])



    if len(left) == 0 or len(right) == 0:
        return make_leaf(data)
    tempA.clear()
    tempB.clear()

    b = tree_maker(pd.DataFrame(right), lenoffeatures)  # Call for left tree
    a = tree_maker(pd.DataFrame(left), lenoffeatures)  # Call for right tree
    tree = node(a, b, False, logRegression, None, len(data))
    return tree

In [10]:
def predict(tree, point):
    predictions = predictResults()
    if tree.isLeaf == True:
        return tree.classes
    lg = tree.model
    side = lg.predict(point)
    if side == 0:
        return predict(tree.left, point)
    else:
        return predict(tree.right, point)

In [15]:
def main():
    df = pd.read_csv('mil_dataset.csv')
    df.head()
    training = df.sample(frac = 0.8, random_state=200)
    testing = df.drop(training.index, axis = 0, inplace=False)
    print("tree training.")
    tree = tree_maker(training, 120)
    print("tree trained.")
    
    
    acc = prediction(tree, testing)
    print("Accuracy = ", acc)
    
main()

tree training.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


leaf


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework

...

leaf
leaf
leaf
leaf
leaf


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework

leaf


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


tree trained.
Accuracy =  56.767582434105044
